In [ ]:
numDimensions = 300
maxSeqLength = 250
batchSize = 24
lstmUnits = 64
numClasses = 2
iterations = 100000
import os
print(os.path.abspath("."))

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
wordsList = np.load(r'/content/drive/MyDrive/LSTM_Sentiment/training_data/wordsList.npy').tolist()
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load(r'/content/drive/MyDrive/LSTM_Sentiment/training_data/wordVectors.npy')

In [ ]:
# import tensorflow as tf
# from tensorflow.python.framework import ops
# ops.reset_default_graph()
# tf.reset_default_graph()
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

# import tensorflow.compat.v1 as tf1
# tf1.disable_v2_behavior() 
# import tensorflow as tf2 #Tensorflow 1.15.2
# from tensorflow.contrib.rnn import LSTMCell

import tensorflow.compat.v1 as tf1
tf1.disable_v2_behavior()
import tensorflow as tf2
from tensorflow import keras


labels = tf2.compat.v1.placeholder(tf2.float32, [batchSize, numClasses])
input_data = tf2.compat.v1.placeholder(tf2.int32, [batchSize, maxSeqLength])

data = tf2.Variable(tf2.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf2.float32)
data = tf2.nn.embedding_lookup(wordVectors,input_data)

lstmCell = tf2.compat.v1.nn.rnn_cell.LSTMCell(lstmUnits)
lstmCell = tf2.compat.v1.nn.rnn_cell.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)
# lstmCell = tf2.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)
value, state = tf2.keras.layers.RNN(lstmCell, data, dtype=tf2.float32)
# value, state =  tf2.compat.v1.nn.dynamic_rnn(lstmCell, data,  dtype=tf2.float32)

weight = tf2.Variable(tf2.truncated_normal([lstmUnits, numClasses]))
# bias = tf2.Variable(tf2.constant(0.1, shape=[numClasses]))
# value = tf2.transpose(value, [1, 0, 2])
# last = tf2.gather(value, int(value.get_shape()[0]) - 1)
# prediction = (tf2.matmul(last, weight) + bias)

# correctPred = tf2.equal(tf2.argmax(prediction,1), tf2.argmax(labels,1))
# accuracy = tf2.reduce_mean(tf2.cast(correctPred, tf2.float32))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.


TypeError: ignored

In [ ]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models'))

In [ ]:
# Removes punctuation, parentheses, question marks, etc., and leaves only alphanumeric characters
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize,maxSeqLength], dtype='int32')
    cleanedSentence = cleanSentences(sentence)
    split = cleanedSentence.split()
    for indexCounter,word in enumerate(split):
        try:
            sentenceMatrix[0,indexCounter] = wordsList.index(word)
        except ValueError:
            sentenceMatrix[0,indexCounter] = 399999 #Vector for unkown words
    return sentenceMatrix

Now, we can create our input text. Try modifying the value for inputText, and see how the outputs change!

In [ ]:
inputText = "That movie was terrible."
inputMatrix = getSentenceMatrix(inputText)

In [ ]:
predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
# predictedSentiment[0] represents output score for positive sentiment
# predictedSentiment[1] represents output score for negative sentiment

if (predictedSentiment[0] > predictedSentiment[1]):
    print("Positive Sentiment")
else:
    print("Negative Sentiment")

NameError: name 'prediction' is not defined

In [ ]:
secondInputText = "That movie was the best one I have ever seen."
secondInputMatrix = getSentenceMatrix(secondInputText)

In [ ]:
predictedSentiment = sess.run(prediction, {input_data: secondInputMatrix})[0]
if (predictedSentiment[0] > predictedSentiment[1]):
    print "Positive Sentiment"
else:
    print "Negative Sentiment"

Positive Sentiment
